<a href="https://colab.research.google.com/github/aledima00/Project4_SemSeg_AML2024/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4 - Semantic Segmentation

First let's download dataset, that is already split in "Train", "Test" and "Val" modules

In [7]:
import os

import logging
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import glob
import numpy as np
from enum import Enum
from PIL import Image
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [2]:
!wget -O "train.zip" "https://zenodo.org/records/5706578/files/Train.zip?download=1"
!unzip "train.zip"

Output streaming troncato alle ultime 5000 righe.
  inflating: Train/Rural/images_png/1040.png  
 extracting: Train/Rural/images_png/1041.png  
 extracting: Train/Rural/images_png/1042.png  
 extracting: Train/Rural/images_png/1043.png  
  inflating: Train/Rural/images_png/1044.png  
 extracting: Train/Rural/images_png/1045.png  
 extracting: Train/Rural/images_png/1046.png  
 extracting: Train/Rural/images_png/1047.png  
 extracting: Train/Rural/images_png/1048.png  
 extracting: Train/Rural/images_png/1049.png  
  inflating: Train/Rural/images_png/105.png  
  inflating: Train/Rural/images_png/1050.png  
  inflating: Train/Rural/images_png/1051.png  
 extracting: Train/Rural/images_png/1052.png  
 extracting: Train/Rural/images_png/1053.png  
  inflating: Train/Rural/images_png/1054.png  
 extracting: Train/Rural/images_png/1055.png  
 extracting: Train/Rural/images_png/1056.png  
 extracting: Train/Rural/images_png/1057.png  
 extracting: Train/Rural/images_png/1058.png  
 extracting

In [3]:
!wget -O "val.zip" "https://zenodo.org/records/5706578/files/Val.zip?download=1"
!unzip "val.zip"

--2024-12-10 15:22:28--  https://zenodo.org/records/5706578/files/Val.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2425958254 (2.3G) [application/octet-stream]
Saving to: ‘val.zip’

val.zip             100%[===================>]   2.26G  27.9MB/s    in 86s     

2024-12-10 15:23:55 (26.8 MB/s) - ‘val.zip’ saved [2425958254/2425958254]

Archive:  val.zip
   creating: Val/
   creating: Val/Rural/
   creating: Val/Rural/images_png/
 extracting: Val/Rural/images_png/2522.png  
 extracting: Val/Rural/images_png/2523.png  
 extracting: Val/Rural/images_png/2524.png  
 extracting: Val/Rural/images_png/2525.png  
 extracting: Val/Rural/images_png/2526.png  
  inflating: Val/Rural/images_png/2527.png  
 extracting: Val/Rural/images_png/2528.png  
  inflating: Val/Rural/images_png/2529.png  
 extracting: Val/Rural/

In [ ]:
!wget -O "Test.zip" "https://zenodo.org/records/5706578/files/Test.zip?download=1"
!unzip "Test.zip"

Create Dataset class and filter urban pictures...

In [4]:
DBG = True

DEVICE = "cpu"
TRAIN_PATH = "Train"
TEST_PATH = "Test"
VAL_PATH = "Val"

BATCH_SIZE = 1 if DBG else 128

In [5]:
Rural_train = os.path.join(TRAIN_PATH, "Rural")
images_rural_train = os.path.join(Rural_train, "images_png")
os.listdir(images_rural_train)

['765.png',
 '473.png',
 '627.png',
 '486.png',
 '461.png',
 '1138.png',
 '707.png',
 '908.png',
 '125.png',
 '691.png',
 '743.png',
 '204.png',
 '273.png',
 '834.png',
 '293.png',
 '292.png',
 '572.png',
 '1302.png',
 '316.png',
 '648.png',
 '305.png',
 '1274.png',
 '120.png',
 '551.png',
 '389.png',
 '638.png',
 '165.png',
 '208.png',
 '650.png',
 '399.png',
 '910.png',
 '487.png',
 '327.png',
 '1060.png',
 '290.png',
 '328.png',
 '553.png',
 '714.png',
 '348.png',
 '235.png',
 '477.png',
 '584.png',
 '1155.png',
 '974.png',
 '882.png',
 '1167.png',
 '124.png',
 '255.png',
 '731.png',
 '499.png',
 '608.png',
 '847.png',
 '244.png',
 '631.png',
 '458.png',
 '31.png',
 '688.png',
 '222.png',
 '449.png',
 '412.png',
 '294.png',
 '303.png',
 '340.png',
 '478.png',
 '1013.png',
 '181.png',
 '1086.png',
 '1198.png',
 '1188.png',
 '19.png',
 '346.png',
 '1108.png',
 '1152.png',
 '1202.png',
 '1203.png',
 '860.png',
 '994.png',
 '318.png',
 '1022.png',
 '403.png',
 '72.png',
 '465.png',
 '12

In [8]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

logger = logging.getLogger(__name__)

class DataType(Enum):
    RURAL = 0
    URBAN = 1

class LoveDA(Dataset):
    def __init__(self, basedir, data_type:DataType, transforms=None, target_transform=None):
        #super(LoveDA, self).__init__(basedir, transforms, target_transform)
        if data_type == DataType.RURAL:
            self.base_path = os.path.join(basedir, "Rural")
        else: #data_type == DataType.URBAN:
            self.base_path = os.path.join(basedir, "Urban")


        # list of integers that identifies paths relative to both images_png and masks_png
        self.int_pathrefs = os.listdir(os.path.join(self.base_path, "images_png"))
        self.int_pathrefs = list(int(st.split(".")[0]) for st in self.int_pathrefs)
        self.transforms = transforms

    def __getitem__(self, idx):
      assert idx < len(self), 'Index out of range'
      image_path = os.path.join(self.base_path, "images_png", str(self.int_pathrefs[idx]) + ".png")
      mask_path = os.path.join(self.base_path, "masks_png", str(self.int_pathrefs[idx]) + ".png")
      image = pil_loader(image_path)
      mask = pil_loader(mask_path)

      if self.transforms is not None:
          image = self.transforms(image)
          mask = self.transforms(mask)
      return image, mask

    def __len__(self):
        return len(self.int_pathrefs)


In [11]:
urban_val = LoveDA(VAL_PATH, DataType.URBAN, transforms=eval_transform)
urban_train = LoveDA(VAL_PATH, DataType.URBAN, transforms=train_transform)

In [12]:
urban_train_dataloader = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
urban_val_dataloader = DataLoader(urban_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [13]:
for elm in urban_train_dataloader:
    print(elm)
    break

[tensor([[[[-0.6627, -0.6627, -0.6392,  ..., -0.5216, -0.5216, -0.5686],
          [-0.6471, -0.6627, -0.6314,  ..., -0.5216, -0.5451, -0.6471],
          [-0.6627, -0.6471, -0.6549,  ..., -0.6000, -0.5922, -0.6549],
          ...,
          [-0.5922, -0.6392, -0.6314,  ..., -0.2392, -0.5843, -0.6392],
          [-0.5686, -0.6235, -0.6314,  ..., -0.2784, -0.6000, -0.6471],
          [-0.5686, -0.6314, -0.6235,  ..., -0.3333, -0.6157, -0.6471]],

         [[-0.5059, -0.5216, -0.4980,  ..., -0.4431, -0.4431, -0.4745],
          [-0.5137, -0.5294, -0.4902,  ..., -0.4196, -0.4510, -0.5216],
          [-0.5216, -0.5373, -0.5373,  ..., -0.4824, -0.4667, -0.5373],
          ...,
          [-0.5059, -0.5294, -0.5059,  ..., -0.1765, -0.4431, -0.4980],
          [-0.4510, -0.4667, -0.4588,  ..., -0.2000, -0.4588, -0.5059],
          [-0.4353, -0.4667, -0.4275,  ..., -0.2392, -0.4824, -0.5137]],

         [[-0.5608, -0.5608, -0.5529,  ..., -0.4824, -0.4824, -0.5216],
          [-0.5529, -0.5686, 

Download the pretrained model with *Resnet101* as backbone

In [ ]:
!wget -O "Pretrained.zip" "http://liangchiehchen.com/projects/released/deeplab_aspp_resnet101/prototxt_and_model.zip"
!unzip "Pretrained.zip" -d "Pretrained"

Archive:  Pretrained.zip
  inflating: Pretrained/train_iter_20000.caffemodel  
  inflating: Pretrained/train2_iter_20000.caffemodel  
  inflating: Pretrained/train.prototxt  
  inflating: Pretrained/test.prototxt  
  inflating: Pretrained/solver2.prototxt  
  inflating: Pretrained/solver.prototxt  
  inflating: Pretrained/init.caffemodel  


Get DeepLabv2 implementation:

In [ ]:
!git clone https://github.com/Gabrysse/MLDL2024_project1.git